<a href="https://colab.research.google.com/github/kukretinishtha/medium_blog/blob/medium/caching_in_vectordb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install chromadb sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 100.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:

In [2]:
import chromadb
from chromadb.config import Settings

# Initialize ChromaDB client
client = chromadb.Client(Settings())
collection = client.get_or_create_collection("semantic_cache_demo")

# Add some documents to ChromaDB
collection.add(
    documents=["How can I help you?", "What's your name?", "Goodbye!"],
    ids=["1", "2", "3"]
)

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:00<00:00, 103MiB/s]


In [3]:
from sentence_transformers import SentenceTransformer
import hashlib
import numpy as np

# Initialize sentence transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate embedding-based cache key
def embed_cache_key(query: str, precision=2):
    embedding = model.encode(query)
    rounded = np.round(embedding, decimals=precision)
    return hashlib.md5(rounded.tobytes()).hexdigest(), embedding.tolist()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import json

# Simulated Redis-style cache
semantic_cache = {}

# Semantic cache-aware vector search
def semantic_vector_search(query: str, top_k=2):
    key, embedding = embed_cache_key(query)

    if key in semantic_cache:
        print(f"Cache hit for '{query}'")
        return json.loads(semantic_cache[key])

    print(f"Cache miss for '{query}' → computing results")
    results = collection.query(query_embeddings=[embedding], n_results=top_k)

    semantic_cache[key] = json.dumps(results)
    return results

In [5]:
# Queries: similar in meaning but different wording
queries = ["How are you?", "What's up?", "What's going on?"]

for q in queries:
    print(f"\n🔍 Query: {q}")
    result = semantic_vector_search(q)
    print(f"📄 Top result: {result['documents'][0][0]}")


🔍 Query: How are you?
Cache miss for 'How are you?' → computing results
📄 Top result: What's your name?

🔍 Query: What's up?
Cache miss for 'What's up?' → computing results
📄 Top result: What's your name?

🔍 Query: What's going on?
Cache miss for 'What's going on?' → computing results
📄 Top result: What's your name?
